In [2]:
import warnings
from fastembed import TextEmbedding

warnings.filterwarnings('ignore')

### Q1 Embedding the query

In [ ]:
query = 'I just discovered the course. Can I join now?'
model_name = "jinaai/jina-embeddings-v2-small-en"

In [18]:
model = TextEmbedding(model_name=model_name)
emb = list(model.embed(query))[0]

print('Size of array:', len(emb))
print('Min value:', min(emb))

Size of array: 512
Min value: -0.11726373885183883


**Q1 Answer:** -0.11

### Q2. Cosine similarity with another vector

In [19]:
import numpy as np
np.linalg.norm(emb)

np.float64(1.0)

In [22]:
query2 = 'Can I still join the course after the start date?'
emb2 = list(model.embed(query2))[0]

In [24]:
print('Cosine similarity between Query and Query2:', emb.dot(emb2))

Cosine similarity between Query and Query2: 0.9008528895674548


**Q2 Answer**: 0.9

### Q3. Ranking by cosine

In [25]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [31]:
idx = 0

for doc in documents:
    v = list(model.embed(doc['text']))[0]
    cosine = emb.dot(v)
    print(f'Cosine between query and document {idx}: ', cosine)
    idx += 1

Cosine between query and document 0:  0.7629684696540238
Cosine between query and document 1:  0.8182378150042889
Cosine between query and document 2:  0.8085397398734189
Cosine between query and document 3:  0.7133079015686243
Cosine between query and document 4:  0.7304499234333598


**Q3 Answer**: document 1

### Q4. Ranking by cosine, version two

In [32]:
idx = 0

for doc in documents:
    v = list(model.embed(doc['question'] + ' ' + doc['text']))[0]
    cosine = emb.dot(v)
    print(f'Cosine between query and document {idx}: ', cosine)
    idx += 1

Cosine between query and document 0:  0.8514543236908068
Cosine between query and document 1:  0.8436594159113068
Cosine between query and document 2:  0.8408287048502558
Cosine between query and document 3:  0.7755157969663908
Cosine between query and document 4:  0.8086007795043938


**Q4 Answer: document 0** - it is becuase question for this document is very similar to query

### Q5. Selecting the embedding model

In [36]:
print('The smallest dimension:', min([model["dim"] for model in TextEmbedding.list_supported_models()]))

The smallest dimension: 384


### Q6 Indexing with qdrant (2 points)

In [37]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [39]:
from qdrant_client import QdrantClient, models
client = QdrantClient("http://localhost:6333")
model_name = "BAAI/bge-small-en"
EMBEDDING_DIMENSIONALITY = 384 #from Q5

In [40]:
# Define the collection name
collection_name = "llm-zoomcamp-homework"

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [42]:
points = []
id = 0

for doc in documents:
    text = doc['question'] + ' ' + doc['text']
    point = models.PointStruct(
        id=id,
        vector=models.Document(text=text, model=model_name),
        payload={
            "text": text,
            "section": doc['section'],
            "course": doc['course']
        } #save all needed metadata fields
    )
    points.append(point)

    id += 1

In [43]:
client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files:  80%|████████  | 4/5 [00:04<00:01,  1.09s/it]
2025-06-30 00:23:00.312 | ERROR    | fastembed.common.model_management:download_model:430 - Could not download model from HuggingFace: [WinError 1314] Klient nie ma wymaganych uprawnień: '..\\..\\blobs\\37fca74771bc76a8e01178ce3a6055a0995f8093' -> 'C:\\Users\\ROG\\AppData\\Local\\Temp\\fastembed_cache\\models--Qdrant--bge-small-en\\snapshots\\8791246cc2a79c7949a4dc0d4a018cbd7d024879\\tokenizer_config.json' Falling back to other sources.
100%|██████████| 77.7M/77.7M [00:03<00:00, 25.4MiB/s]


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [44]:
len(points)

375

In [46]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_name 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [47]:
result = search(query)

In [48]:
result.points[0].payload

{'text': 'The course has already started. Can I still join it? Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
 'section': 'General course-related questions',
 'course': 'machine-learning-zoomcamp'}

In [49]:
result.points[0].score

0.8703172

**Q6 Answer:** 0.87